In [0]:
%sh

pip uninstall -y rtree
sudo apt install -y libspatialindex-dev
pip install rtree
pip install pygeos

In [0]:
import warnings

warnings.filterwarnings("ignore")

In [0]:
%run ./Setup

In [0]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [0]:
import geopandas as gpd
from shapely import wkt

df1 = (spark.read
         .format("singlestore")
         .load("hot_routes.bakerloo_stops")
      )

bakerloo_stops = df1.toPandas()
bakerloo_stops = gpd.GeoDataFrame(bakerloo_stops)
bakerloo_stops["geometry"] = bakerloo_stops["geometry"].apply(wkt.loads)
bakerloo_stops.set_crs(epsg = 4326, inplace = True)

In [0]:
stops = bakerloo_stops.plot(color = "black", markersize = 15)

stops.set_axis_off()

stops.plot()

In [0]:
df2 = (spark.read
         .format("singlestore")
         .load("hot_routes.bakerloo_sections")
      )

bakerloo_sections = df2.toPandas()
bakerloo_sections = gpd.GeoDataFrame(bakerloo_sections)
bakerloo_sections["geometry"] = bakerloo_sections["geometry"].apply(wkt.loads)
bakerloo_sections.set_crs(epsg = 4326, inplace = True)

In [0]:
sections = bakerloo_sections.plot(color = "#B36305", linewidth = 3)

sections.set_axis_off()

sections.plot()

In [0]:
base = bakerloo_sections.plot(color = "#B36305", linewidth = 3)

bakerloo_stops.plot(ax = base, color = "black", markersize = 15)

base.set_axis_off()

base.plot()

In [0]:
df3 = (spark.read
         .format("singlestore")
         .load("hot_routes.crimes")
      )

crimes = df3.toPandas()
crimes = gpd.GeoDataFrame(crimes)
crimes["geometry"] = crimes["geometry"].apply(wkt.loads)
crimes.set_crs(epsg = 4326, inplace = True)

In [0]:
btp_crimes = crimes.plot()

btp_crimes.set_axis_off()

btp_crimes.plot()

In [0]:
df4 = (spark.read
         .format("singlestore")
         .load("hot_routes.bakerloo_line_buff")
      )

bakerloo_line_buff = df4.toPandas()
bakerloo_line_buff = gpd.GeoDataFrame(bakerloo_line_buff)
bakerloo_line_buff["geometry"] = bakerloo_line_buff["geometry"].apply(wkt.loads)
bakerloo_line_buff.set_crs(epsg = 4326, inplace = True)

In [0]:
base = bakerloo_line_buff.plot(
  color = "lightgrey",
  edgecolor = "black",
  figsize = (10, 10)
)

base.set_axis_off()

base.plot()

In [0]:
crimes = bakerloo_line_buff.overlay(
  crimes, how = "intersection", keep_geom_type = False
)

In [0]:
crimes

In [0]:
base = bakerloo_line_buff.plot(
  color = "lightgrey",
  edgecolor = "black",
  figsize = (10, 10)
)

bakerloo_sections.plot(ax = base, color = "#B36305", linewidth = 3)

crimes.plot(ax = base, color = "red")

bakerloo_stops.plot(ax = base, color = "black", markersize = 15)

base.set_axis_off()

base.plot()

In [0]:
bline_segments = gpd.sjoin_nearest(crimes, bakerloo_sections)

bline_segments.rename(
  columns = {"index_right" : "segment"}, inplace = True
)

In [0]:
bline_segments.head()

In [0]:
sections_freq = gpd.GeoDataFrame(
  bline_segments["segment"].value_counts()
)

sections_freq.rename(
  columns = {"segment" : "freq"}, inplace = True
)

In [0]:
sections_freq

In [0]:
bakerloo_sections = bakerloo_sections.join(sections_freq)

bakerloo_sections["freq"].fillna(0, inplace = True)

In [0]:
bakerloo_sections.head()

In [0]:
midpoint_crimes = bakerloo_sections["freq"].mean()

In [0]:
midpoint_crimes

In [0]:
base = bakerloo_line_buff.plot(
  color = "lightgrey",
  edgecolor = "black",
  figsize = (10, 10)
)

bakerloo_sections.plot(
  ax = base,
  column = "freq",
  linewidth = 3,
  cmap = "OrRd",
  legend = True,
  legend_kwds = {"label" : "Number of crimes", "orientation" : "horizontal"}
)

bakerloo_stops.plot(ax = base, color = "black", markersize = 15)

base.set_axis_off()

base.plot()

In [0]:
bakerloo_sections.to_crs(epsg = 3310, inplace = True)

In [0]:
bakerloo_sections["length"] = bakerloo_sections["geometry"].length

In [0]:
bakerloo_sections.length.sum()

In [0]:
bakerloo_sections.to_crs(epsg = 4326, inplace = True)

In [0]:
bakerloo_sections.head()

In [0]:
bakerloo_sections["crime_per_m"] = bakerloo_sections["freq"] / bakerloo_sections["length"]

In [0]:
bakerloo_sections.head()

In [0]:
base = bakerloo_line_buff.plot(
  color = "lightgrey",
  edgecolor = "black",
  figsize = (10, 10)
)

bakerloo_sections.plot(
  ax = base,
  column = "crime_per_m",
  linewidth = 3 + bakerloo_sections["crime_per_m"] * 100,
  cmap = "OrRd",
  legend = True,
  legend_kwds = {"label" : "Rate of crimes per metre", "orientation" : "horizontal"}
)

bakerloo_stops.plot(ax = base, color = "black", markersize = 15)

base.set_axis_off()

base.plot()

In [0]:
import folium

from folium import plugins
from folium.plugins import HeatMap
from branca.colormap import linear

London = [51.509865, -0.118092]

m = folium.Map(location = London, tiles = "Stamen Terrain", control_scale = True, zoom_start = 14)

folium.GeoJson(
  bakerloo_line_buff
).add_to(folium.FeatureGroup(name = "Buffer").add_to(m))

HeatMap(
  data = list(zip(crimes.geometry.y, crimes.geometry.x))
).add_to(folium.FeatureGroup(name = "Crime Heat Map").add_to(m))

colormap = linear.OrRd_09.scale(
  bakerloo_sections.crime_per_m.min(),
  bakerloo_sections.crime_per_m.max()
)
colormap.caption = "Rate of crimes per metre"
colormap.add_to(m)

for i in range(0, len(bakerloo_sections)):
  x, y = bakerloo_sections.geometry.iloc[i].coords.xy
  folium.PolyLine(
    locations = [(y[0], x[0]), (y[1], x[1])],
    color = colormap(bakerloo_sections["crime_per_m"][i]),
    weight = 3 + bakerloo_sections["crime_per_m"][i] * 100,
    opacity = 1
  ).add_to(m)

stations_group = folium.FeatureGroup(name = "Stations").add_to(m)

for i in range(0, len(bakerloo_stops)):
  stations_group.add_child(folium.Marker(
    icon = folium.Icon(
      color = "red",
      icon = "train",
      icon_color = "white",
      prefix = "fa"
    ),
    location = [bakerloo_stops.geometry[i].y, bakerloo_stops.geometry[i].x],
    popup = bakerloo_stops.stn_name[i],
  ))

crimes_group = folium.FeatureGroup(name = "Crimes").add_to(m)

for i in range(0, len(crimes)):
  crimes_group.add_child(folium.Marker(
    icon = folium.Icon(icon = "info-sign"),
    location = [crimes.geometry[i].y, crimes.geometry[i].x],
    popup = crimes["crime_type"][i]
  ))

folium.LayerControl().add_to(m)

plugins.Fullscreen(
  position = "topright",
  title = "Fullscreen",
  title_cancel = "Exit",
  force_separate_button = True
).add_to(m)

m